In [ ]:
!pip install pandas seaborn xlsxwriter openpyxl scikit-learn==1.0.2

## Import libraries
We are using a new library called scikit-learn, originally created and released for free by researchers at the French national laboratory INRIA.

In [ ]:
import numpy as np
import pandas as pd

import sklearn.pipeline
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.model_selection
import sklearn.metrics

## Load data
__For lab: make a copy of [this sheet in Google Sheets](https://docs.google.com/spreadsheets/d/1IfMPD-PnYqS_fZiZVCElBz_BovOulMYj7cco8ssNf-I/edit?usp=sharing) and rename it with your PID (`A9999999_happysad`) in Google Sheets__



__For lab: in Google Sheets, add bias for and/or against a particular group by adding at least 10 new rows__

__For lab: download the spreadsheet as Excel and upload it to this folder. Make sure you rename the filename in the code cell below to match the one you downloaded and uploaded__

In [ ]:
data = pd.read_excel("A9999999_happysad.xlsx")

In [ ]:
data

In [ ]:
data['output'].value_counts()

## Filtering out neutral sentences

Sometimes we only care about some of the cases or categories in our dataset. What if we want to only make a happy/sad classifier, and not a happy/neutral/sad classifier? We can use query, and ask it to return all casses where the output is not equal (`!=`) to `'neutral'`. Note that the entire query has to be inside a double quote, and then if we are asking it to compare text, the text has to be inside single quotes. If `output` was a number, we might instead do `data.query("output != 0")`.

In [ ]:
data.query("output != 'neutral'")

In [ ]:
data_happysad = data.query("output != 'neutral'")

In [ ]:
data_happysad['output'].value_counts()

## Split into training and testing datasets
We want to randomly select 80% of this data to use to train the model, then use the remaining 20% to test how good the model is on examples it has not seen before.

In [ ]:
input_train, input_test, output_train, output_test = sklearn.model_selection.train_test_split(data_happysad['input'], 
                                                                                             data_happysad['output'], 
                                                                                             test_size=0.2)

## Create a blank model from a three-part pipeline and then train it


In [ ]:
model = sklearn.pipeline.Pipeline([
    ('vect', sklearn.feature_extraction.text.CountVectorizer()),
    ('tfidf', sklearn.feature_extraction.text.TfidfTransformer()),
    ('clf', sklearn.naive_bayes.MultinomialNB()),
])


In [ ]:
model.fit(input_train, output_train)

## Testing the model on the other 20% of data
Remember the `input_test` dataset? We will use the `model.predict()` function to score those 1035 emails.

In [ ]:
output_predicted = model.predict(input_test)

We can use `model.score()` by first inputting the 'true' labels, then the predictions. We get a percentage of the model's __accuracy__:

$accuracy = \frac{\mbox{number of correct predictions}}{\mbox{total number of items predicted}}$

In [ ]:
print(model.score(output_test, output_predicted))

# The confusion matrix

In [ ]:
confusion_matrix = sklearn.metrics.confusion_matrix(output_test, output_predicted, labels=model.classes_)

In [ ]:
display = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix, display_labels=model.classes_)
display.plot()


## If you have more than 2 categories, it might be more useful to look at the classification report:

In [ ]:
print(sklearn.metrics.classification_report(output_test, output_predicted))

# For lab: auditing for a new kind of bias

__For lab: replace these countries with those from the kind of group you introduced a bias for/against. Be sure to include examples where you did not introduce a bias!__

In [ ]:
audit_categories = ["France", "Germany", "the United States", "China"]

__For lab: change the line `sample_text = "Yay! I won a vacation to " + example + "!"` so that it makes sense for the kind of bias you are testing against__

In [ ]:
results_list = []

for example in audit_categories:

    sample_text = "Yay! I won a vacation to " + example + "!"
    probability = model.predict_proba([sample_text])[0][0]
    
    result = {'example':example,
              'happy_prediction':probability}
    
    results_list.append(result)

In [ ]:
data_audit = pd.DataFrame(results_list)
data_audit.sort_values('happy_prediction')

# Saving results of audit to excel

In [ ]:
data_audit.sort_values('happy_prediction').to_excel("audit_results.xlsx")